<a href="https://colab.research.google.com/github/R12942159/DeepLearning/blob/main/DLCV_p1_inferance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !gdown https://drive.google.com/file/d/1-1IQCvKSFwMoDzHjfkMH4H1AirA4lesG/view?usp=share_link -O EFFICIENTNET_best_model.pth

In [ ]:
import torch


# Get cuda from GPU device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
import os
import argparse

parser = argparse.ArgumentParser()
parser.add_argument("path1", help="display a square of a given number")
parser.add_argument("path2", help="display a square of a given number")
args = parser.parse_args()

img_paths = args.path1

# search file paths
img_paths_val = [os.path.join(img_paths, i) for i in os.listdir(img_paths) if i.endswith('.png')]

In [ ]:
import torch
from PIL import Image
import numpy as np


class PretrainDataset(torch.utils.data.Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform

    def __len__(self):
        """number of samples"""
        return len(self.img_paths)

    def __getitem__(self, idx):
        """read 1 sample"""
        # Read img
        path = self.img_paths[idx] # get img path
        img = Image.open(path).convert('RGB') # read img

        # transform img
        img = self.transform(img)

        # Read file name
        file_name = path.split('/')[-1]

        return img, file_name

In [ ]:
from torchvision.models import efficientnet_b4, EfficientNet_B4_Weights


# Preprocess Transform
transform =  EfficientNet_B4_Weights.DEFAULT.transforms()

# Build Dataset
val_ds = PretrainDataset(img_paths_val, transform)

In [ ]:
# Build DataLoaders
BATCH_SIZE = 50
val_loader = torch.utils.data.DataLoader(val_ds, BATCH_SIZE, shuffle=False)

In [ ]:
# Build Model and Replace classifier
from torchvision import models


model = models.efficientnet_b4(weights=EfficientNet_B4_Weights.DEFAULT)

last_layer = list(model.children())[-1]
weights = last_layer[1].weight
bias = last_layer[1].bias
new_linear_layer = torch.nn.Linear(in_features=weights.shape[1], out_features=50, bias=True)
new_linear_layer.weight.data = weights
new_linear_layer.bias.data = bias
last_layer[1] = new_linear_layer

model.load_state_dict(torch.load("EFFICIENTNET_best_model.pth"))

<All keys matched successfully>

In [ ]:
model = model.to(device) # move model to GPU
model.eval() # Set model to evaluation mode

In [ ]:
# Create temp tensor
file_name = []
y_pred_logits = torch.tensor([]).to(device)

In [ ]:
from torch import nn


# deactivate autograd engine and reduce memory usage and speed up computations
with torch.no_grad():
    for batch_i, (x, f_name) in enumerate(val_loader):
        x = x.to(device)

        pred = model(x)
        y_pred_logits = torch.cat((y_pred_logits, pred), dim=0)
        file_name.extend(f_name)

y_pred_cls = y_pred_logits.argmax(1) # get class idx with max prob (100, 3) -> (100, )

file_name = np.array(file_name, dtype=str)

# Move tensor to CPU and convert to np.ndarray
y_pred_cls = y_pred_cls.cpu().numpy()

In [ ]:
import csv

pred_csv = args.path2
# pred_csv = '/content/drive/MyDrive/DLCV/pred.csv'

try:
    os.makedirs(os.path.dirname(pred_csv), exist_ok=True)
except:
    pass

with open(pred_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(('filename', 'label'))
    for data in zip(file_name, y_pred_cls):
        writer.writerow(data)